# Apresentação:

O objetivo desse código é explocar as capacidades da API da Hugging Face.

In [1]:
# Importando classe pipeline
from transformers import pipeline

# Bibliotecas de suporte:
import pandas as pd

In [2]:
# Texto utilizado:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

# Classificação de Texto:

Apesar do tópico ser nomeado no livro como **classificação de texto**, o exemplo em questão trata de uma **análise de sentimentos**, um caso particular de classificação de texto, porém relevante o suficiente para ser pontuado por si só. Então vale esse _disclaimer_, apesar do nome, o que estamos fazendo aqui não é classificação de texto, é análise de sentimento.

In [3]:
# Instanciando Classificador de Texto:
classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\amori\anaconda3\envs\bndes\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Classificando Texto:
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


Observe que, para tarefas de análise de sentimentos, o pipeline só retorna um dos rótulos POSITIVO ou NEGATIVO, já que o outro pode ser inferido ao calcular **1-score**. O nada mais é do que uma forma de complementar a pontuação de um modelo em tarefas de **classificação binária**, como a **análise de sentimentos**. Ele indica que a probabilidade atribuída ao rótulo alternativo (não previsto pelo modelo) pode ser calculada subtraindo a pontuação dada ao rótulo previsto de 1.

# Reconhecimento de Entidade Nomeada:

No **NLP**, objetos do mundo real, como produtos, lugares e pessoas, são chamados de **entidades nomeadas**, e extraí-las do texto é chamado de **reconhecimento de entidades nomeadas (_Named Entity Recognition_ - NER)**. Podemos aplicar o **NER** carregando o `pipeline` correspondente e fornecendo a ele nossa avaliação de cliente.

In [5]:
# Instanciando classe:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\amori\anaconda3\envs\bn

In [6]:
# Reconhecendo Entidades Nomeadas: 
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590255,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


Pode-se ver que o `pipeline` detectou todas as entidades e também atribuiu uma categoria, como **ORG** (organização), **LOC** (localização) ou **PER** (pessoa) a cada uma delas. Aqui, usamos o atributo `aggregation_strategy` para agrupar as palavras de acordo com as previsões do modelo. Por exemplo, a entidade "Optimus Prime" é composta de duas palavras, mas é atribuída a uma única categoria: **MISC** (diversos). 

As pontuações nos dizem o quão confiante o modelo estava sobre as entidades que identificou. Podemos ver que ele estava menos confiante sobre "Decepticons" e a primeira ocorrência de "Megatron", ambos os quais ele falhou em agrupar como uma única entidade.

Note aqueles símbolos _hash_ estranhos (#) na coluna de palavras da tabela anterior? Estes são produzidos pelo modelo
tokenizer, que divide palavras em unidades atômicas chamadas tokens.


# Resposta a perguntas (_Question Answering_):

Na tarefa de **resposta às perguntas** (_Question Answering_), fornecemos ao modelo uma passagem de texto chamada **contexto**, junto
com uma pergunta cuja resposta gostaríamos de extrair. O modelo então retorna a extensão do texto correspondente à resposta. Esse approach talvez lembre um pouco o **RAG (_Retrieval-Augmented Generation_)**.


## Similaridades entre QA e RAG:

* Fornecimento de um contexto: Tanto no QA tradicional quanto no RAG, um contexto é fornecido para que o modelo possa responder à pergunta. Esse contexto pode ser um documento, uma passagem ou informações recuperadas de uma base de dados.
  
* Extração de respostas: Em ambos os casos, o objetivo é fornecer uma resposta baseada em informações relevantes do contexto. No QA, a resposta pode ser um trecho de texto específico extraído diretamente do contexto, enquanto no RAG, o modelo pode gerar respostas com base em informações recuperadas.

## Diferenças:
* Question Answering: No QA tradicional, o modelo trabalha diretamente com o contexto fornecido e tenta extrair uma resposta específica, normalmente como um "span" (trecho) de texto. O modelo não gera novo conteúdo, mas apenas localiza a resposta dentro do texto existente.

* RAG (Retrieval-Augmented Generation): RAG combina recuperação de documentos (Retrieval) com geração de texto (Generation). Primeiro, o modelo busca informações relevantes em grandes bases de dados ou fontes externas e, em seguida, gera uma resposta com base nesses documentos recuperados. A principal diferença aqui é que o RAG pode buscar informações além do contexto direto e, depois, gerar uma resposta baseada nesses múltiplos documentos.

In [7]:
# Instanciando Modelo:
reader = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\amori\anaconda3\envs\bndes\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
# Realizando Pergunta:
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


O pipeline também retornou números inteiros de início e fim que correspondem aos índices de caracteres onde o trecho da resposta foi encontrado (assim como acontece com a marcação de **NER**). Existem várias abordagens para _question answerin_ que será investigada em notebook próprio, mas este tipo específico é chamado de **_question answering extrativo_**, pois a resposta é extraída diretamente do texto.

# Sumarização (_Summarization_):

O objetivo da sumarização de texto é pegar um texto longo como entrada e gerar uma versão curta com todos os **fatos relevantes**. Esta é uma tarefa muito mais complicada do que as anteriores, pois exige que o modelo gere um texto coerente. Seguindo um padrão que já deve ser familiar, podemos instanciar um pipeline de sumarização da seguinte maneira:

In [9]:
# Instanciando modelo:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
C:\Users\amori\anaconda3\envs\bndes\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Gerando inferência do modelo:
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

Your min_length=56 must be inferior than your max_length=45.
C:\Users\amori\anaconda3\envs\bndes\lib\site-packages\transformers\generation\utils.py:1282: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead.


Esse resumo não está nada mal! Embora partes do texto original tenham sido copiadas, o modelo conseguiu captar a essência do problema e identificar corretamente que "Bumblebee" (que apareceu no final) era o autor da reclamação. Neste exemplo, você também pode ver que passamos alguns argumentos como `max_length` e `clean_up_tokenization_spaces` para o `pipeline`; esses permitem ajustar as saídas em tempo de execução.

# Tradução:

Assim como a sumarização, a tradução é uma tarefa onde a resposta consistem geração de texto. Diga-se de passagem, a arquitetura transformers nasce para essa tarefa, tradução de textos. Os pesquisadores pensaram o modelo para isso, e outras pessoas foram trabalhando a arquitetura para outras tarefas até chegar nesse monte de tarefas que temos hoje.

In [17]:
# Instanciando Tradutor:
translator = pipeline("translation_en_to_de",model="Helsinki-NLP/opus-mt-en-de")

In [18]:
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.


Novamente, o modelo produziu uma tradução muito boa que utilizou corretamente os pronomes formais em alemão, como "Ihrem" e "Sie". Aqui também mostramos como você pode substituir o modelo padrão no pipeline para escolher o mais adequado para sua aplicação — e você pode encontrar modelos para milhares de pares de idiomas no _Hugging Face Hub_.

# Geração de Texto (_Text Generation_):

A grande sereja do bolo, a geração de texto.

In [19]:
# Instanciando Modelo:
generator = pipeline("text-generation")

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\amori\anaconda3\envs\bndes\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amori\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [24]:
# Gerando Prompt:
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response

# Inferência do modelo:
outputs = generator(prompt, max_length=300)
print(outputs[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I am an experienced collector of Transformers toys and have been

acquainted with many many Autobot figures, and will gladly assist me in finding a new

product before Christmas. I am fully on board with the Transformers 5.5 in-house line of figures. I

have a special place in my heart for the Optimus Prime Collection, and would happily exchange any

customer I will have on sale with it.

As